In [1]:
#Efficientnet_attention Definition and Compilation
import tensorflow as tf
from keras.layers import GlobalAveragePooling2D, Bidirectional, LSTM,  Dense,  Reshape, MultiHeadAttention, LayerNormalization, Add, TimeDistributed, Lambda
from keras.models import Model
from keras.applications import EfficientNetB0

def create_efficientnet_bilstm_attention(input_shape=(128, 431, 1), lstm_units=64, num_heads=8, projection_dim=64):

     # Create an EfficientNetB0 model without the top layers as a feature extractor
    feature_extractor = EfficientNetB0(include_top=False, weights=None, input_shape=input_shape)

    # Pass the input through the feature extractor to obtain features
    x = feature_extractor.output

    # Apply global average pooling to reduce the spatial dimensions of the features
    x = GlobalAveragePooling2D()(x)

    # Pad the input tensor with zeros to match the desired shape
    x = tf.pad(x, paddings=[[0, 0], [0, 13]])

    # Reshape the tensor to match the required input shape for the LSTM layer
    x = Reshape((431, 3))(x)

    # Add a bidirectional LSTM layer to capture sequential information
    x = Bidirectional(LSTM(lstm_units, return_sequences=True))(x)

    # Implement multi-head self-attention using dense layers for query, key, and value
    query = Dense(projection_dim)(x)
    key = Dense(projection_dim)(x)
    value = Dense(projection_dim)(x)

    # Apply multi-head attention with the specified number of heads and key dimensions
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(query, key, value)

    # Add a dense layer to match the dimensions of the input tensor and the attention output
    attention = Dense(x.shape[-1])(attention)

    # Add the attention output to the input tensor and apply layer normalization
    attention = LayerNormalization(epsilon=1e-6)(Add()([attention, x]))

    # Add a time-distributed dense layer with sigmoid activation for prediction
    x = TimeDistributed(Dense(1, activation='sigmoid'))(attention)

    # Squeeze the last dimension of the tensor to obtain the final output shape
    x = Lambda(lambda x: tf.squeeze(x, axis=-1))(x)

    # Return the complete model with the feature extractor as input and the predictions as output
    return Model(inputs=feature_extractor.input, outputs=x)

#Load the function into memory
model = create_efficientnet_bilstm_attention()

#Print the summary
model.summary()

#Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 431, 1  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 431, 1)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 128, 431, 1)  3           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 129, 433, 1)  0           ['normalization[0][0]']      

In [ ]:
#Example of how to import OCSED_util
from OCSED_util import OCSED_util
util = OCSED_util()

In [ ]:
#Example of how to load your saved model
save_dir = util.current_dir / 'Trained_models'
model = util.load_trained_model(save_dir, "baseline_AT_CRNN")

In [ ]:
# Example of how to get already split training data
X_train, X_val, y_train, y_val = util.preprocess_data()

In [ ]:
#Example of how to train you defined custom model
hist = util.trainer.train_model(model=model, 
                                X_train=X_train, 
                                y_train=y_train, 
                                X_val=X_val, 
                                y_val=y_val, 
                                epochs=100, 
                                batch_size=16, 
                                patience=10,
                                eval_metric=util.get_eval_f1_metric())

In [ ]:
#Example of how to save your trained custom model
save_dir = util.current_dir / 'Trained_models'
util.save_model(model, save_dir, "baseline_AT_CRNN_v2")

In [ ]:
#Example of how to evaluate your loaded or trained model
util.update_threshold(0.5) #Change as desired for testing
util.eval_model(model)